In [3]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd

# 參數初始化xavier initialization  
根據某一層網路的輸入、輸出節點數量自動調整最合適的分佈。

In [4]:
def xavier_init(fan_in,fan_out,constant=1):
    low=-constant*np.sqrt(6.0/(fan_in+fan_out))
    high=constant*np.sqrt(6.0/(fan_in+fan_out))
    return tf.random_uniform((fan_in,fan_out),
                              minval=low,maxval=high,
                              dtype=tf.float32)


In [5]:
class AdditiveGaussianNoiseAutoencoder(object):
    def __init__(self,n_input,n_hidden,transfer_function=tf.nn.softplus,optimizer=tf.train.AdamOptimizer(),scale=0.1):
        self.n_input=n_input
        self.n_hidden=n_hidden
        self.transfer=transfer_function
        self.scale=tf.placeholder(tf.float32)
        self.training_scale=scale
        network_weights=self._initialize_weights()
        self.weights=network_weights
        
        #接下來定義網路結構
        self.x=tf.placeholder(tf.float32,[None,self.n_input])
        self.hidden=self.transfer(
            tf.add(
                tf.matmul(
                    self.x+scale*tf.random_normal((n_input,)),
                    self.weights['w1']
                ),
                self.weights['b1']
            )
        )
        self.reconstruction=tf.add(tf.matmul(self.hidden,self.weights['w2']),self.weights['b2'])
        
        #定義自編碼器的損失函數
        self.cost=0.5*tf.reduce_sum(tf.pow(tf.subtract(self.reconstruction,self.x),2.0))
        self.optimizer=optimizer.minimize(self.cost)
        
        init=tf.global_variables_initializer()
        self.sess=tf.Session()
        self.sess.run(init)
    def _initialize_weights(self):
        all_weights=dict()
        all_weights['w1']=tf.Variable(xavier_init(self.n_input,self.n_hidden))
        all_weights['b1']=tf.Variable(tf.zeros([self.n_hidden],dtype=tf.float32))
        all_weights['w2']=tf.Variable(tf.zeros([self.n_hidden,self.n_input],dtype=tf.float32))
        all_weights['b2']=tf.Variable(tf.zeros([self.n_input],dtype=tf.float32))

        return all_weights
    
    def partial_fit(self,X):
        cost,opt=self.sess.run((self.cost,self.optimizer),feed_dict={self.x:X,self.scale:self.training_scale})
        return cost
    
    def calc_total_cost(self,X):
        return self.sess.run(
            self.cost,
            feed_dict={
                self.x:X,
                self.scale:self.training_scale
            }
        )
    def transform(self,X):
        return self.sess.run(
            self.hidden,
            feed_dict={
                self.x:X,
                self.scale:self.training_scale
            }
        )
    def generate(self,hidden=None):
        if hidden is None:
            hidden=np.random.normal(size=self.weights['b1'])
        return self.sess.run(
            self.reconstruction,
            feed_dict={
                self.hidden:hidden
            }
        )
    
    def reconstruction(self,X):
        return self.sess.run(
            self.reconstruction,
            feed_dict={
                self.x:X,
                self.scale:self.training_scale

            }
        )
    
    def getWeights(self):
        return self.sess.run(self.weights['w1'])
    def getBiase(self):
        return self.sess.run(self.weights['b1'])

In [7]:
mnist=input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [8]:
def standard_scale(X_train,X_test):
    preprocessor=prep.StandardScaler().fit(X_train)
    X_train=preprocessor.transform(X_train)
    X_test=preprocessor.transform(X_test)
    return X_train,X_test

In [9]:
def get_random_block_from_data(data,batch_size):
    start_index=np.random.randint(0,len(data)-batch_size)
    return data[start_index:(start_index+batch_size)]



In [10]:
X_train,X_test=standard_scale(mnist.train.images,mnist.test.images)

In [12]:
n_samples=int(mnist.train.num_examples)
training_epochs=20
batch_size=128
display_step=1

In [20]:
autoencoder=AdditiveGaussianNoiseAutoencoder(
    n_input=784,
    n_hidden=200,
    transfer_function=tf.nn.softplus,
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001),
    scale=0.01
)

In [24]:
#Training前參數
weight_before_train=autoencoder.getWeights()
weight_before_train

array([[-0.03378165, -0.07176102, -0.06751537, ..., -0.05130841,
         0.04958694,  0.04304562],
       [ 0.07529439,  0.01881208, -0.00796723, ...,  0.05667144,
        -0.07747819, -0.0312485 ],
       [-0.05013537,  0.03296241,  0.03774371, ...,  0.03569144,
        -0.02456729,  0.00708638],
       ...,
       [ 0.05199704,  0.07471754, -0.07079258, ...,  0.01330276,
         0.05338037, -0.03771444],
       [ 0.01574226,  0.00107655, -0.02561682, ...,  0.04942307,
        -0.01527624,  0.0040527 ],
       [-0.00791927,  0.04179253,  0.00279534, ..., -0.02732766,
         0.02824192,  0.02886745]], dtype=float32)

In [17]:
for epoch in range(training_epochs):
    avg_cost=0.
    total_batch=int(n_samples/batch_size)
    for i in range(total_batch):
        batch_xs=get_random_block_from_data(X_train,batch_size)
        
        cost=autoencoder.partial_fit(batch_xs)
        
        avg_cost += cost/n_samples*batch_size
    if epoch % display_step==0:
        print("Epoch:",'%04d' % (epoch+1),'cost=',"{:.9f}".format(avg_cost))

Epoch: 0001 cost= 18617.321873864
Epoch: 0002 cost= 12126.012306818
Epoch: 0003 cost= 10527.090507386
Epoch: 0004 cost= 10099.338019318
Epoch: 0005 cost= 9080.782606250
Epoch: 0006 cost= 9426.867237500
Epoch: 0007 cost= 9850.560368750
Epoch: 0008 cost= 9285.220034091
Epoch: 0009 cost= 9657.277647727
Epoch: 0010 cost= 8593.236076705
Epoch: 0011 cost= 8403.773209091
Epoch: 0012 cost= 8224.721195455
Epoch: 0013 cost= 8571.384389205
Epoch: 0014 cost= 7723.577074432
Epoch: 0015 cost= 8393.180772159
Epoch: 0016 cost= 8820.796188636
Epoch: 0017 cost= 7765.981620455
Epoch: 0018 cost= 8018.703921591
Epoch: 0019 cost= 8659.253739205
Epoch: 0020 cost= 7672.779822159


In [18]:
print('Total cost:' + str(autoencoder.calc_total_cost(X_test)))

Total cost:658773.8


In [31]:
encode_feature=autoencoder.transform(X_train)

In [32]:
encode_feature

array([[0.17124765, 0.6783413 , 0.36384264, ..., 0.36981454, 1.1977699 ,
        0.05044066],
       [1.5294659 , 1.7466848 , 1.5379183 , ..., 0.9434279 , 2.2056053 ,
        1.7927393 ],
       [0.59827197, 1.0346497 , 0.20885886, ..., 0.50986814, 0.71495426,
        0.23694426],
       ...,
       [0.9562799 , 1.6139101 , 0.51452017, ..., 1.0998256 , 1.4699045 ,
        0.8800701 ],
       [0.38236818, 0.76742613, 0.11375509, ..., 0.5895511 , 0.82838523,
        1.5679029 ],
       [0.20375349, 1.5139087 , 2.28573   , ..., 0.7209778 , 0.1800679 ,
        0.34302384]], dtype=float32)

In [19]:
autoencoder.getWeights()

array([[ 0.02256429,  0.05932727, -0.06663494, ..., -0.09210242,
         0.0240842 ,  0.02897671],
       [ 0.11438754,  0.03031569,  0.06131024, ...,  0.08560225,
         0.12691982, -0.00927777],
       [-0.01298253,  0.01128005,  0.02308381, ...,  0.00652111,
         0.03105803, -0.03866806],
       ...,
       [-0.02450373,  0.05649551,  0.10894255, ..., -0.06524988,
        -0.04503337,  0.09875945],
       [ 0.00816757,  0.01973294, -0.07212574, ..., -0.12473406,
        -0.04937341, -0.010747  ],
       [-0.0430139 ,  0.031868  ,  0.01997165, ...,  0.04978435,
        -0.01059116,  0.01385012]], dtype=float32)